In [2]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 初始化加载器（以《西游记》在线阅读页面为例）
loader = WebBaseLoader("https://www.gutenberg.org/cache/epub/23962/pg23962-images.html")  # 替换为目标URL
docs = loader.load()

# 提取文本内容与元数据
content=docs[0].page_content# 网页正文



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# 查看第一个文档（前100字符）
print(docs[0].page_content[:100])


The Project Gutenberg eBook of 西遊記, by Cheng'en Wu


















The Project Gutenberg eBook of


In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
!pip install sentence-transformers

In [28]:
!pip install hf_xet

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 MB 15.6 MB/s eta 0:00:00m eta 0:00:010:00:01


In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
vectorstore  = Chroma.from_documents(
     documents=all_splits,
    collection_name="xiyouji",
    embedding=HuggingFaceEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"  # 支持中英文的轻量级模型‌:ml-citation{ref="6,7" data="citationList"}
    ),
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)

/tmp/ipykernel_6564/3666754692.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=HuggingFaceEmbeddings(
/root/anaconda3/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# 使用 VectorStoreRetriever 从向量存储中检索与查询最相关的文档
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [8]:
# 替换为 DeepSeek 的 API 参数
llm = ChatOpenAI(
    model="deepseek-chat",  # DeepSeek 指定的模型名称
    base_url="https://api.deepseek.com",  # DeepSeek 的 API 端点
    api_key="sk-3e5ff44e82b745a7ab7a748b806951c2",  # 替换为你的 DeepSeek API Key
    temperature=0.5,
    max_tokens=4000
)

In [9]:
# 使用 hub 模块拉取 rag 提示词模板
prompt = hub.pull("rlm/rag-prompt")

/root/anaconda3/envs/langchain/lib/python3.11/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
# 定义格式化文档的函数
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [11]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
# 流式生成回答
for chunk in rag_chain.stream("西游记有几个人"):
    print(chunk, end="", flush=True)

《西游记》中取经的主要人物共有四人：唐僧（玄奘）、孙悟空、猪八戒和沙僧。

In [13]:
for chunk in rag_chain.stream("西游记有多少个妖怪"):
    print(chunk, end="", flush=True)

根据提供的上下文，无法准确统计《西游记》中妖怪的总数。文中提到“百十大小头目”和众多小妖，但具体数量未明确说明。建议查阅全书或相关统计资料以获取准确数字。

In [14]:
for chunk in rag_chain.stream("西游记的最后一回讲的是什么"):
    print(chunk, end="", flush=True)

西游记的最后一回讲的是唐僧师徒历经九九八十一难后终于到达西天取得真经，如来佛祖封唐僧为旃檀功德佛、孙悟空为斗战胜佛、猪八戒为净坛使者、沙僧为金身罗汉，白马也封为八部天龙。师徒四人功德圆满，返回东土大唐传扬佛法。

In [15]:
from langchain_core.prompts import PromptTemplate
template = """请基于以下背景信息回答问题，并严格遵守回答规则：
【背景信息】
{context}
【回答规则】
1. 如果背景信息不包含答案，直接回答"该问题不在知识库范围内"
2. 回答需分点说明，使用「1. 2. 3.」的格式
3. 每点不超过20个汉字，总长度控制在1000字内
4. 在回答末尾添加与问题相关的拓展建议
5. 使用口语化中文，避免专业术语

问题：{question}

请参考以下回答格式：
【要点总结】
1. 核心要点1
2. 核心要点2
【拓展建议】相关建议内容
"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain2 = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [20]:
# 流式生成回答
for chunk in rag_chain2.stream("西游记主角几个人"):
    print(chunk, end="", flush=True)

【要点总结】
1. 唐僧师徒四人
2. 孙悟空、猪八戒、沙僧
3. 白龙马也算成员

【拓展建议】可以看看原著了解详细人物关系

In [21]:
for chunk in rag_chain2.stream("西游记前十回有多少妖怪"):
    print(chunk, end="", flush=True)

【要点总结】
1. 背景信息未提前十回
2. 仅描述当前回目妖怪
3. 无法统计总数

【拓展建议】
建议查阅原著前十回目录
或相关统计资料获取准确数据

In [22]:
for chunk in rag_chain2.stream("西游记的最后一回讲的是什么"):
    print(chunk, end="", flush=True)

【要点总结】
1. 背景信息未提最后一回
2. 仅包含狮驼岭等片段
3. 无取经结局相关内容

【拓展建议】建议查阅原著第100回或电视剧结局了解唐僧师徒取得真经、封佛等最终情节